In [1]:
import yaml
config = yaml.load(open("config.yaml", "r"), Loader=yaml.FullLoader)
print(config)

{'batch_size': 512, 'warm_up': 10, 'epochs': 100, 'load_model': 'None', 'eval_every_n_epochs': 1, 'save_every_n_epochs': 5, 'log_every_n_steps': 50, 'fp16_precision': False, 'init_lr': 0.0005, 'weight_decay': '1e-5', 'gpu': 'cuda:0', 'model_type': 'gin', 'model': {'num_layer': 5, 'emb_dim': 300, 'feat_dim': 512, 'drop_ratio': 0, 'pool': 'mean'}, 'aug': 'node', 'dataset': {'num_workers': 12, 'valid_size': 0.05, 'data_path': 'data/pubchem-10m-clean.txt'}, 'loss': {'temperature': 0.1, 'use_cosine_similarity': True}}


In [15]:
import torch
a=torch.randn(4,1)
b=torch.rand(4,1)
c=torch.cat((a,b),dim=1)
c.size()

torch.Size([4, 2])

In [16]:
from utils.nt_xent import NTXentLoss
# Assume we have a batch of 2D embeddings (for simplicity)
zis = torch.randn(10, 2)  # 10 samples, each with a 2D embedding
zjs = torch.randn(10, 2)  # 10 samples, each with a 2D embedding

# Initialize the NTXentLoss module
nt_xent_loss = NTXentLoss(device='cpu', batch_size=10, temperature=0.5, use_cosine_similarity=True)

# Compute the loss
loss = nt_xent_loss(zis, zjs)

# print(f"NT-Xent Loss: {loss.item()}")


NT-Xent Loss: 3.8967463970184326


In [19]:
import torch
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self, batch_size, temperature, device):
        super(MyModel, self).__init__()
        self.batch_size = batch_size
        self.temperature = temperature
        self.device = device
        self.criterion = nn.CrossEntropyLoss()
        self.mask_samples_from_same_repr = self._create_mask()

    def _create_mask(self):
        # Create a mask to filter out the scores from the positive samples
        mask = torch.ones((2 * self.batch_size, 2 * self.batch_size))
        mask = mask.fill_diagonal_(0)
        return mask.bool()

    def similarity_function(self, representations1, representations2):
        # Define your similarity function here
        return torch.mm(representations1, representations2.t())

    def forward(self, representations):
        similarity_matrix = self.similarity_function(representations, representations)

        l_pos = torch.diag(similarity_matrix, self.batch_size)
        r_pos = torch.diag(similarity_matrix, -self.batch_size)
        positives = torch.cat([l_pos, r_pos]).view(2 * self.batch_size, 1)

        negatives = similarity_matrix[self.mask_samples_from_same_repr].view(2 * self.batch_size, -1)

        logits = torch.cat((positives, negatives), dim=1)
        logits /= self.temperature

        labels = torch.zeros(2 * self.batch_size).to(self.device).long()
        loss = self.criterion(logits, labels)

        return loss

# Initialize your model
model = MyModel(batch_size=32, temperature=0.1, device='cpu')

# Create some dummy data for representations
representations = torch.randn(64, 128)

# Run the model
loss = model(representations)
print(loss)


tensor(269.2440)
